In [1]:
import pandas
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pandas.read_csv('train.csv')

In [3]:
df

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
data = np.array(df)
m,n = data.shape
data_dev = data[0:1000]
data_train = data[1000:m]
X_train = data_train[:,1:]
y_train = data_train[:,0]
X_dev = data_dev[:,1:]
y_dev = data_dev[:,0]



In [11]:
def parameeters():
    w1 = np.random.randn(10,784)
    w2 = np.random.randn(10,10)
    b1 = np.random.randn(10,1)
    b2 = np.random.randn(10,1)
    return w1,w2,b1,b2

def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

def Relu(X):
    return np.maximum(0,X)

def forward_propagation(X,w1,w2,b1,b2):
    Z1 = np.dot(w1,X.T) + b1
    A1 = Relu(Z1)
    Z2 = np.dot(w2,A1) + b2
    A2 = softmax(Z2)
    return Z1,A1,Z2,A2

def one_hot(y):
    one_hot_Y = np.zeros((y.size, y.max()+1))
    one_hot_Y[np.arange(y.size), y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y
    
def back_prop(X,y,w1,w2,b1,b2,Z1,A1,Z2,A2,learning_rate):

    m = X.shape[0]
    one_hot_Y = one_hot(y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1/m * np.dot(dZ2,A1.T)
    db2 = 1/m * np.sum(dZ2,axis=1,keepdims=True)
    dZ1 = np.dot(w2.T,dZ2) * (Z1 > 0)
    dW1 = 1/m * np.dot(dZ1,X)
    db1 = 1/m * np.sum(dZ1,axis=1,keepdims=True)
    return w1,w2,b1,b2

def regularization(w1,w2,lambd,m):
    w1 = w1 - (lambd/m) * w1
    w2 = w2 - (lambd/m) * w2
    return w1,w2

def cost_function(A2,y):
    m = y.shape[0]
    cost = -1/m * np.sum(y*np.log(A2))
    return cost

def predict(X,w1,w2,b1,b2):
    Z1 = np.dot(w1,X.T) + b1
    A1 = Relu(Z1)
    Z2 = np.dot(w2,A1) + b2
    A2 = softmax(Z2)
    return A2

def accuracy(y_pred,y):
    y_pred = y_pred.T
    y = y.T
    m = y.shape[0]
    correct = 0
    for i in range(m):
        if np.argmax(y_pred[i]) == np.argmax(y[i]):
            correct += 1
    return correct/m

def Adam(X_train, y_train, alpha, epochs, epsilon, beta1, beta2):
    w1,w2,b1,b2 = parameeters()
    m = X_train.shape[0]
    vdw1 = np.zeros(w1.shape)
    vdw2 = np.zeros(w2.shape)
    sdw1 = np.zeros(w1.shape)
    sdw2 = np.zeros(w2.shape)
    vdb1 = np.zeros(b1.shape)
    vdb2 = np.zeros(b2.shape)
    sdb1 = np.zeros(b1.shape)
    sdb2 = np.zeros(b2.shape)
    for i in range(epochs):

        Z1,A1,Z2,A2 = forward_propagation(X_train,w1,w2,b1,b2)
        dw1,dw2,db1,db2 = back_prop(X_train,y_train,w1,w2,b1,b2,Z1,A1,Z2,A2,alpha)
        vdw1 = beta1 * vdw1 + (1-beta1) * dw1
        vdw2 = beta1 * vdw2 + (1-beta1) * dw2
        vdb1 = beta1 * vdb1 + (1-beta1) * db1
        vdb2 = beta1 * vdb2 + (1-beta1) * db2
        sdw1 = beta2 * sdw1 + (1-beta2) * (dw1**2)
        sdw2 = beta2 * sdw2 + (1-beta2) * (dw2**2)
        sdb1 = beta2 * sdb1 + (1-beta2) * (db1**2)
        sdb2 = beta2 * sdb2 + (1-beta2) * (db2**2)
        vdw1_corrected = vdw1 / (1 - beta1**i)
        vdw2_corrected = vdw2 / (1 - beta1**i)
        vdb1_corrected = vdb1 / (1 - beta1**i)
        vdb2_corrected = vdb2 / (1 - beta1**i)
        sdw1_corrected = sdw1 / (1 - beta2**i)
        sdw2_corrected = sdw2 / (1 - beta2**i)
        sdb1_corrected = sdb1 / (1 - beta2**i)
        sdb2_corrected = sdb2 / (1 - beta2**i)
        w1 = w1 - alpha * vdw1_corrected / (np.sqrt(sdw1_corrected) + epsilon)
        w2 = w2 - alpha * vdw2_corrected / (np.sqrt(sdw2_corrected)+ epsilon)
        b1 = b1 - alpha * vdb1_corrected / (np.sqrt(sdb1_corrected) + epsilon)
        b2 = b2 - alpha * vdb2_corrected / (np.sqrt(sdb2_corrected)+ epsilon)
        cost = cost_function(A2,y_train)
        if i % 100 == 0:
            print(cost)
    return w1,w2,b1,b2

In [12]:
w1,w2,b1,b2 = Adam(X_train, y_train, 0.01, 1000, 1e-8, 0.9, 0.999)

C:\Users\karan\AppData\Local\Temp\ipykernel_4572\1542037873.py:9: RuntimeWarning: overflow encountered in exp
  return np.exp(x) / np.sum(np.exp(x), axis=0)
C:\Users\karan\AppData\Local\Temp\ipykernel_4572\1542037873.py:9: RuntimeWarning: invalid value encountered in divide
  return np.exp(x) / np.sum(np.exp(x), axis=0)
C:\Users\karan\AppData\Local\Temp\ipykernel_4572\1542037873.py:89: RuntimeWarning: divide by zero encountered in divide
  vdw1_corrected = vdw1 / (1 - beta1**i)
C:\Users\karan\AppData\Local\Temp\ipykernel_4572\1542037873.py:90: RuntimeWarning: divide by zero encountered in divide
  vdw2_corrected = vdw2 / (1 - beta1**i)
C:\Users\karan\AppData\Local\Temp\ipykernel_4572\1542037873.py:91: RuntimeWarning: divide by zero encountered in divide
  vdb1_corrected = vdb1 / (1 - beta1**i)
C:\Users\karan\AppData\Local\Temp\ipykernel_4572\1542037873.py:92: RuntimeWarning: divide by zero encountered in divide
  vdb2_corrected = vdb2 / (1 - beta1**i)
C:\Users\karan\AppData\Local\Temp\

nan


KeyboardInterrupt: 